* Python code replication of:
" https://www.kaggle.com/victorchernozhukov/analyzing-rct-reemployment-experiment "
* Created by: Alexander Quispe and Anzony Quispe 

# Analyzing RCT data with Precision Adjustment

## Data

In this lab, we analyze the Pennsylvania re-employment bonus experiment, which was previously studied in "Sequential testing of duration data: the case of the Pennsylvania ‘reemployment bonus’ experiment" (Bilias, 2000), among others. These experiments were conducted in the 1980s by the U.S. Department of Labor to test the incentive effects of alternative compensation schemes for unemployment insurance (UI). In these experiments, UI claimants were randomly assigned either to a control group or one of five treatment groups. Actually, there are six treatment groups in the experiments. Here we focus on treatment group 4, but feel free to explore other treatment groups. In the control group the current rules of the UI applied. Individuals in the treatment groups were offered a cash bonus if they found a job within some pre-specified period of time (qualification period), provided that the job was retained for a specified duration. The treatments differed in the level of the bonus, the length of the qualification period, and whether the bonus was declining over time in the qualification period; see http://qed.econ.queensu.ca/jae/2000-v15.6/bilias/readme.b.txt for further details on data. 
  

In [14]:
!pip install pyreadr

In [18]:
import pandas as pd
import pyreadr

In [19]:
## loading the data
Penn = pd.read_csv("../../../data/penn_jae.dat" , sep='\s', engine='python')
n = Penn.shape[0]
p_1 = Penn.shape[1]
Penn = Penn[ (Penn['tg'] == 4) | (Penn['tg'] == 0) ]

In [20]:
Penn.shape

(5099, 24)

In [21]:
#this columns were not dropped out :  Unnamed: 13, recall
Penn.columns
Penn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5099 entries, 0 to 13911
Data columns (total 24 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   abdt         5099 non-null   int64  
 1   tg           5099 non-null   int64  
 2   inuidur1     5099 non-null   int64  
 3   inuidur2     5099 non-null   int64  
 4   female       5099 non-null   int64  
 5   black        5099 non-null   int64  
 6   hispanic     5099 non-null   int64  
 7   othrace      5099 non-null   int64  
 8   dep          5099 non-null   int64  
 9   q1           5099 non-null   int64  
 10  q2           5099 non-null   int64  
 11  q3           5099 non-null   int64  
 12  q4           5099 non-null   int64  
 13  Unnamed: 13  5099 non-null   int64  
 14  q5           5099 non-null   int64  
 15  q6           5099 non-null   int64  
 16  recall       5099 non-null   int64  
 17  agelt35      5099 non-null   int64  
 18  agegt54      5099 non-null   int64  
 19  durab

In [22]:
# Dependent variable
Penn['T4'] = (Penn[['tg']]==4).astype(int)

# Create category variable
Penn['dep'] = Penn['dep'].astype( 'category' )
Penn.head()

,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1,...,q6,recall,agelt35,agegt54,durable,nondurable,lusd,husd,muld,T4
0,10824,0,18,18,0,0,0,0,2,0,...,0,0,0,0,0,0,1,0,NaN,0
3,10824,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,NaN,0
4,10747,0,27,27,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,NaN,0
11,10607,4,9,9,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,NaN,1
12,10831,0,27,27,0,0,0,0,1,0,...,0,0,1,1,0,1,0,0,NaN,0


In [23]:
Penn['dep'].unique()

[2, 0, 1]
Categories (3, int64): [0, 1, 2]

### Model 
To evaluate the impact of the treatments on unemployment duration, we consider the linear regression model:

$$
Y =  D \beta_1 + W'\beta_2 + \varepsilon, \quad E \varepsilon (D,W')' = 0,
$$

where $Y$ is  the  log of duration of unemployment, $D$ is a treatment  indicators,  and $W$ is a set of controls including age group dummies, gender, race, number of dependents, quarter of the experiment, location within the state, existence of recall expectations, and type of occupation.   Here $\beta_1$ is the ATE, if the RCT assumptions hold rigorously.


We also consider interactive regression model:

$$
Y =  D \alpha_1 + D W' \alpha_2 + W'\beta_2 + \varepsilon, \quad E \varepsilon (D,W', DW')' = 0,
$$
where $W$'s are demeaned (apart from the intercept), so that $\alpha_1$ is the ATE, if the RCT assumptions hold rigorously.

Under RCT, the projection coefficient $\beta_1$ has
the interpretation of the causal effect of the treatment on
the average outcome. We thus refer to $\beta_1$ as the average
treatment effect (ATE). Note that the covariates, here are
independent of the treatment $D$, so we can identify $\beta_1$ by
just linear regression of $Y$ on $D$, without adding covariates.
However we do add covariates in an effort to improve the
precision of our estimates of the average treatment effect.

### Analysis

We consider 

*  classical 2-sample approach, no adjustment (CL)
*  classical linear regression adjustment (CRA)
*  interactive regression adjusment (IRA)

and carry out robust inference using the *estimatr* R packages. 

# Carry out covariate balance check

This is done using "lm_robust" command which unlike "lm" in the base command automatically does the correct Eicher-Huber-White standard errors, instead othe classical non-robus formula based on the homoscdedasticity command.

In [24]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import numpy as np

In [25]:
formula1 = "T4~(female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2"

model_results = smf.ols(formula1, data = Penn).fit().get_robustcov_results(cov_type = "HC1")


print(model_results.summary())
print( "Number of regressors in the basic model:",len(model_results.params), '\n')

                            OLS Regression Results                            
Dep. Variable:                     T4   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     26.59
Date:                Tue, 26 Apr 2022   Prob (F-statistic):               0.00
Time:                        08:08:52   Log-Likelihood:                -3360.7
No. Observations:                5099   AIC:                             6941.
Df Residuals:                    4989   BIC:                             7660.
Df Model:                         109                                         
Covariance Type:                  HC1                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.3890    

C:\Users\Roberto\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 119, but rank is 109
  warnings.warn('covariance of constraints does not have full '


## Regress treatment on all covariates using data from R

In [26]:
y = Penn[['T4']].reset_index( drop = True )

In [27]:
# Get data from R
result = pyreadr.read_r('../../../data/m_reg.RData')
X_vars = result['X1']

In [28]:
# model = "T4~(female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2"

model_results = sm.OLS( y, X_vars ).fit().get_robustcov_results(cov_type = "HC1")

print(model_results.summary())
print( "Number of regressors in the basic model:",len(model_results.params), '\n')

                            OLS Regression Results                            
Dep. Variable:                     T4   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     29.07
Date:                Tue, 26 Apr 2022   Prob (F-statistic):               0.00
Time:                        08:08:52   Log-Likelihood:                -3358.5
No. Observations:                5099   AIC:                             6923.
Df Residuals:                    4996   BIC:                             7596.
Df Model:                         102                                         
Covariance Type:                  HC1                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
(Intercept)              0.3215 

We see that that even though this is a randomized experiment, balance conditions are failed.

# Model Specification

In [29]:
# model specifications
# take log of inuidur1
Penn["log_inuidur1"] = np.log( Penn["inuidur1"] ) 
log_inuidur1 = pd.DataFrame(np.log( Penn["inuidur1"] ) ).reset_index( drop = True )

# no adjustment (2-sample approach)
formula_cl = 'log_inuidur1 ~ T4'

ols_cl = smf.ols( formula = formula_cl, data = Penn ).fit().get_robustcov_results(cov_type = "HC1")

# adding controls

formula_cra = 'log_inuidur1 ~ T4 + (female+black+othrace+C(dep)+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2'
# Omitted dummies: q1, nondurable, muld

ols_cra = smf.ols( formula = formula_cra, data = Penn ).fit().get_robustcov_results(cov_type = "HC1")


print(ols_cl.summary())
print(ols_cra.summary())
print( "Number of regressors in the control model:",len(ols_cra.params), '\n')

                            OLS Regression Results                            
Dep. Variable:           log_inuidur1   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.680
Date:                Tue, 26 Apr 2022   Prob (F-statistic):             0.0172
Time:                        08:08:53   Log-Likelihood:                -8223.8
No. Observations:                5099   AIC:                         1.645e+04
Df Residuals:                    5097   BIC:                         1.646e+04
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.0568      0.021     98.156      0.0

C:\Users\Roberto\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 120, but rank is 110
  warnings.warn('covariance of constraints does not have full '


# Model Specification using data from R.

In [30]:
# model specifications
# take log of inuidur1

log_inuidur1 = pd.DataFrame(np.log( Penn["inuidur1"] ) ).reset_index( drop = True )

# no adjustment (2-sample approach)
formula_cl = 'log_inuidur1 ~ T4'

# adding controls
# formula_cra = 'log_inuidur1 ~ T4 + (female+black+othrace+dep+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)**2'
# Omitted dummies: q1, nondurable, muld

ols_cl = smf.ols( formula = formula_cl, data = Penn ).fit().get_robustcov_results(cov_type = "HC1")

#getting data
# Get data from R
result = pyreadr.read_r('../../../data/ols_cra_reg.RData')
X_vars = result['X1']

ols_cra = sm.OLS( log_inuidur1, X_vars ).fit().get_robustcov_results(cov_type = "HC1")

# Results 
print(ols_cl.summary())
print(ols_cra.summary())
print( "Number of regressors in the interactive model:",len(ols_cra.params), '\n')

                            OLS Regression Results                            
Dep. Variable:           log_inuidur1   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.680
Date:                Tue, 26 Apr 2022   Prob (F-statistic):             0.0172
Time:                        08:08:53   Log-Likelihood:                -8223.8
No. Observations:                5099   AIC:                         1.645e+04
Df Residuals:                    5097   BIC:                         1.646e+04
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.0568      0.021     98.156      0.0

The interactive specificaiton corresponds to the approach introduced in Lin (2013).

In [31]:
# create Y variable 
log_inuidur1 = pd.DataFrame(np.log( Penn["inuidur1"] )).reset_index( drop = True )

In [32]:
# Reset index to estimation
# Get data from R
result = pyreadr.read_r('../../../data/ols_ira_reg.RData')
X_vars = result['S1']

ols_ira = sm.OLS( log_inuidur1, X_vars ).fit().get_robustcov_results(cov_type = "HC1")

# Results 
print(ols_ira.summary())

                            OLS Regression Results                            
Dep. Variable:               inuidur1   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     30.32
Date:                Tue, 26 Apr 2022   Prob (F-statistic):               0.00
Time:                        08:08:53   Log-Likelihood:                -8016.3
No. Observations:                5099   AIC:                         1.644e+04
Df Residuals:                    4896   BIC:                         1.777e+04
Df Model:                         202                                         
Covariance Type:                  HC1                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
(Intercept)     

C:\Users\Roberto\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 202, but rank is 197
  warnings.warn('covariance of constraints does not have full '


# Next we try out partialling out with lasso

In [33]:
#pip install multiprocess

In [34]:
import hdmpy

Next we try out partialling out with lasso

In [35]:
# Get data from R
result = pyreadr.read_r('../../../data/rlasso_ira_reg.RData')
X_vars = result['S']

In [36]:
X_vars

,T4,X(Intercept),Xfemale,Xblack,Xothrace,Xfactor(dep)1,Xfactor(dep)2,Xq2,Xq3,Xq4,...,T4:Xagelt35:agegt54,T4:Xagelt35:durable,T4:Xagelt35:lusd,T4:Xagelt35:husd,T4:Xagegt54:durable,T4:Xagegt54:lusd,T4:Xagegt54:husd,T4:Xdurable:lusd,T4:Xdurable:husd,T4:Xlusd:husd
0,-0.342224,0.0,-0.404001,-0.121985,-0.007256,-0.112179,0.836242,-0.203765,-0.235536,-0.225927,...,-0.0,0.024699,0.045236,0.042887,0.006175,0.010940,0.009061,0.018591,0.009598,-0.0
1,-0.342224,0.0,-0.404001,-0.121985,-0.007256,-0.112179,-0.163758,-0.203765,-0.235536,-0.225927,...,-0.0,0.024699,0.045236,0.042887,0.006175,0.010940,0.009061,0.018591,0.009598,-0.0
2,-0.342224,0.0,-0.404001,-0.121985,-0.007256,-0.112179,-0.163758,-0.203765,-0.235536,0.774073,...,-0.0,0.024699,0.045236,0.042887,0.006175,0.010940,0.009061,0.018591,0.009598,-0.0
3,0.657776,0.0,-0.404001,-0.121985,-0.007256,-0.112179,-0.163758,-0.203765,0.764464,-0.225927,...,0.0,-0.047472,-0.086947,-0.082432,-0.011868,-0.021027,-0.017415,-0.035733,-0.018447,0.0
4,-0.342224,0.0,-0.404001,-0.121985,-0.007256,0.887821,-0.163758,-0.203765,-0.235536,-0.225927,...,-0.0,0.024699,0.045236,0.042887,-0.336049,-0.331284,0.009061,-0.323633,0.009598,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5094,0.657776,0.0,-0.404001,-0.121985,-0.007256,-0.112179,-0.163758,-0.203765,0.764464,-0.225927,...,0.0,-0.047472,-0.086947,-0.082432,-0.011868,-0.021027,-0.017415,-0.035733,-0.018447,0.0
5095,0.657776,0.0,-0.404001,-0.121985,-0.007256,-0.112179,0.836242,0.796235,-0.235536,-0.225927,...,0.0,-0.047472,-0.086947,-0.082432,-0.011868,-0.021027,-0.017415,-0.035733,-0.018447,0.0
5096,-0.342224,0.0,-0.404001,-0.121985,-0.007256,-0.112179,0.836242,0.796235,-0.235536,-0.225927,...,-0.0,0.024699,-0.296988,0.042887,0.006175,0.010940,0.009061,0.018591,0.009598,-0.0
5097,0.657776,0.0,-0.404001,-0.121985,-0.007256,-0.112179,-0.163758,-0.203765,-0.235536,-0.225927,...,0.0,-0.047472,-0.086947,-0.082432,-0.011868,-0.021027,-0.017415,-0.035733,-0.018447,0.0


In [37]:
result = hdmpy.rlassoEffects( X_vars, log_inuidur1, index = 0 )       

rlasso_ira = pd.DataFrame(np.array( (result.res['coefficients'][0] , result.res['se'][0] , \
           result.res['t'][0] , result.res['pval'][0] ) ).reshape(1, 4) , columns = ['Coef.' , \
                            "Std.Err." , "t" , 'P>|t|'] , index = ['T4'])

In [38]:
# result.res is a object dictionary

result.res.keys()

dict_keys(['coefficients', 'se', 't', 'pval', 'lasso_regs', 'index', 'samplesize', 'residuals', 'coef_mat', 'selection_matrix'])

In [39]:
result.res['coefficients']

,0
T4,-0.078886


In [40]:
rlasso_ira

,Coef.,Std.Err.,t,P>|t|
T4,-0.078886,0.03553,-2.220297,0.026399


### Results

In [41]:
table2 = np.zeros((2, 4))
table2[0,0] = ols_cl.summary2().tables[1]['Coef.']['T4']
table2[0,1] = ols_cra.summary2().tables[1]['Coef.']['T4']
table2[0,2] = ols_ira.summary2().tables[1]['Coef.']['T4']
table2[0,3] = rlasso_ira['Coef.']['T4']

table2[1,0] = ols_cl.summary2().tables[1]['Std.Err.']['T4']
table2[1,1] = ols_cra.summary2().tables[1]['Std.Err.']['T4']
table2[1,2] = ols_ira.summary2().tables[1]['Std.Err.']['T4']
table2[1,3] = rlasso_ira['Std.Err.']['T4']

table2 = pd.DataFrame(table2, columns = ["CL", "CRA", "IRA", "IRA Lasso"], \
                      index = ["estimate","standard error"])

table2

,CL,CRA,IRA,IRA Lasso
estimate,-0.085455,-0.079680,-0.075501,-0.078886
standard error,0.035856,0.035591,0.035605,0.035530


In [42]:
table2.to_latex
print(table2.to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &        CL &       CRA &       IRA &  IRA Lasso \\
\midrule
estimate       & -0.085455 & -0.079680 & -0.075501 &  -0.078886 \\
standard error &  0.035856 &  0.035591 &  0.035605 &   0.035530 \\
\bottomrule
\end{tabular}



Treatment group 4 experiences an average decrease of about $7.8\%$ in the length of unemployment spell.


Observe that regression estimators delivers estimates that are slighly more efficient (lower standard errors) than the simple 2 mean estimator, but essentially all methods have very similar standard errors. From IRA results we also see that there is not any statistically detectable heterogeneity.  We also see the regression estimators offer slightly lower estimates -- these difference occur perhaps to due minor imbalance in the treatment allocation, which the regression estimators try to correct.


